In [40]:
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import math
import pandas as pd
import numpy as np
import streamlit as st
from joblib import dump 
import joblib




df = pd.read_csv('dataset2.csv')

print(df.head(), df.shape)
# separation des features et de la variable cible
X = df.drop('charges', axis=1)
y = df[['charges']]
print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {y.shape}''')


# division du dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])
print(f''' verif du split 80 20 
80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
# 20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')


   age     sex     bmi  children smoker     region      charges bmi_category
0   19  female  27.900         0    yes  southwest  16884.92400   Overweight
1   18    male  33.770         1     no  southeast   1725.55230        Obese
2   28    male  33.000         3     no  southeast   4449.46200        Obese
3   33    male  22.705         0     no  northwest  21984.47061       Normal
4   32    male  28.880         0     no  northwest   3866.85520   Overweight (1337, 8)
verif des dimensions X et Y
      X (dataset sans la variable cible): (1337, 7)
      Y (la variable cible) : (1337, 1)
 verif du split 80 20 
80% du dataset : X train -> (1136, 7), Y train -> (1136, 1)
# 20% du dataset : X test -> (201, 7), Y test -> (201, 1)


In [41]:
#Elasticnet
# préprocesseur pour les variables numériques
def log_transform(x):
    return np.log(x + 1)

log_transformer = FunctionTransformer(log_transform)

preprocessor_num = Pipeline(steps=[
    ('log', FunctionTransformer(log_transform)),
])

preprocessor_cat = Pipeline(steps=[
    ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessor_num, ['age', 'bmi', 'children']),
        ('cat', preprocessor_cat, ['region', 'sex', 'smoker'])
    ]
)

pipeline_lasso = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('regression', Lasso())
])

#test de différents hyperparam d'alpha pour le lasso
param_grid_lasso = {
    'regression__alpha': np.arange(7,10,0.1),
    'regression__precompute': [True],
    'regression__max_iter': [5000],
    
}

### grid search
grid_lasso = GridSearchCV(pipeline_lasso, param_grid_lasso, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_lasso.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_lasso.predict(X_test)
# print(y_pred)





/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15051342.55940628, tolerance: 13974848.585028803
  model = cd_fast.enet_coordinate_descent_gram(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14985321.298412323, tolerance: 13974848.585028803
  model = cd_fast.enet_coordinate_descent_gram(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14917651.966957092, tolerance: 13974848.585028803
  model = cd_fast.enet_coordinate_descent_gram(
/home/utilisateur/Bureau/ml/lib/python3.10/site

In [42]:
#test des les differents metriques sur modele elasticnet
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle elasticnet: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle elasticnet: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle elasticnet: {rmse} \n rappel moyenne charge : 13279')

Mean Squared Error du modèle elasticnet: 12162660.945019629
Coefficient de determination R² du modèle elasticnet: 0.9160708363237503
Root Mean Squared Error (RMSE) du modèle elasticnet: 3487.5006731210287 
 rappel moyenne charge : 13279


In [43]:
best_model = grid_lasso.best_estimator_
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('log',
                                                                   FunctionTransformer(func=<function log_transform at 0x7f7100851c60>))]),
                                                  ['age', 'bmi', 'children']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['region', 'sex', 'smoker',
                                                   'bmi_category'])])),
                ('poly', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('regression',
                 Lasso(alpha=7.0, max_iter=5000, precompute=True))])

In [44]:
# extration  modèle Lasso
lasso_model = best_model.named_steps['regression']

# obtention des coefficients
coefficients = lasso_model.coef_
coefficients

for idx, coef in enumerate(sorted(coefficients, key=abs, reverse=True)):
    print(f"Coefficient {idx + 1}: {coef}")

Coefficient 1: 19138.29953972337
Coefficient 2: -15672.827894972763
Coefficient 3: -9807.643222156894
Coefficient 4: -3707.2385825352467
Coefficient 5: 3427.4006967419555
Coefficient 6: -3234.7065318910522
Coefficient 7: 1659.0511833642768
Coefficient 8: 1176.074586810933
Coefficient 9: -991.5191315144527
Coefficient 10: -802.1449374272572
Coefficient 11: 659.1587630947782
Coefficient 12: 384.5953009685923
Coefficient 13: 379.1933782110389
Coefficient 14: 371.26217254924046
Coefficient 15: -346.4968159053674
Coefficient 16: -280.5207460279142
Coefficient 17: 267.5284813480147
Coefficient 18: 259.61248207614807
Coefficient 19: -223.02449361689838
Coefficient 20: 168.77328394628367
Coefficient 21: 161.88759515678552
Coefficient 22: -145.07729541192808
Coefficient 23: -144.5025080004376
Coefficient 24: -139.5039636699542
Coefficient 25: 139.0609778624716
Coefficient 26: -136.60015412752813
Coefficient 27: 135.02899748496347
Coefficient 28: 128.06652528056642
Coefficient 29: 116.9395042119

In [45]:
new_data = pd.DataFrame({
    'age': [35, 40, 30, 60,60],  
    'sex': ['female', 'male', 'male', 'female','female'],  
    'bmi': [22.5, 30.0, 25.0, 47,50],
    'children': [1, 2, 0, 5,5],
    'smoker': ['yes', 'no', 'no', 'yes','yes'],  
    'region': ['southwest', 'northeast', 'southeast', 'southeast', 'southeast'],
})

# faire des prédictions avec le modèle optimisé
new_y_ped = best_model.predict(new_data)

# ajouter les prédictions à new_data
new_data['predicted_charges'] = new_y_ped

print(new_data)

KeyError: "['bmi_category'] not in index"

In [ ]:
import pickle
with open('best_model.pkl', 'wb') as fichier:
    pickle.dump(best_model, fichier)